In [1]:
# For Training Model

from keras.engine import Model
from keras.layers import *
from keras.applications import *
from keras.optimizers import Adam

from keras import layers

import glob

# Data
# X = input images [num, row, cols, channel]
# y = input controls 2D matrix [throttle value, steering angle]

# resolution for images (row, col, channel)
image_dims=(600,800,1)

# Build model
#----------------------------------------------------------------------------------------------------------
inputs=Input(shape=image_dims)

# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# add last layers
layer = Flatten()(x)
layer = Dense(1024, activation='relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1024, activation='relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1024, activation='relu')(layer)
layer = Dropout(0.5)(layer)

# output controls. it will be Linear function
output_layer = Dense(2)(layer)

model = Model(inputs, output_layer)

#----------------------------------------------------------------------------------------------------------

# compile the model
try:
    model.load_weights('model/xception_net_segmented_steering_throttle.h5')
except(OSError):
    print('Model not present! Training from scratch')

adam = Adam(lr=1e-5, decay=0.5)
model.compile(optimizer=adam, loss='mse')

print('Model loaded!')

# load controls
y=np.load('data/controls.npy')
y=y.reshape(20,1000,2)

# train for data batches
filelist = glob.glob('data/training_batch_1*.npy')

i=0
for epoch in range(10):
    for file in filelist:
        X=np.load(file)
        print('\nNew X loaded! Size: ',X.shape)
        model.fit(x=X, y=y[i], epochs=1, batch_size=2)
        print('Model trained for X.')
        model.save('model/xception_net_segmented_steering_throttle.h5')
        print('Model Saved!')
        i+=1

X_test=np.load('data/training_batch_1.19.npy')
y_test=y[19]

print('Test loss: ', model.evaluate(x=X_test,y=y_test,batch_size=2))

Using TensorFlow backend.


Model loaded!
X loaded! Size:  (1000, 600, 800, 1)
Epoch 1/1
1000/1000 [==============================] - 174s - loss: 0.1963   
Model trained for X.
Model Saved!
X loaded! Size:  (1000, 600, 800, 1)
Epoch 1/1
1000/1000 [==============================] - 172s - loss: 0.1441   
Model trained for X.
Model Saved!
X loaded! Size:  (1000, 600, 800, 1)
Epoch 1/1
1000/1000 [==============================] - 172s - loss: 0.1271   
Model trained for X.
Model Saved!
X loaded! Size:  (1000, 600, 800, 1)
Epoch 1/1
1000/1000 [==============================] - 172s - loss: 0.1108   
Model trained for X.
Model Saved!
X loaded! Size:  (1000, 600, 800, 1)
Epoch 1/1
1000/1000 [==============================] - 172s - loss: 0.1293   
Model trained for X.
Model Saved!
X loaded! Size:  (1000, 600, 800, 1)
Epoch 1/1
1000/1000 [==============================] - 172s - loss: 0.1192   
Model trained for X.


KeyboardInterrupt: 

In [10]:
# For generating commands

# from keras.engine import Model
# from keras.layers import *
# from keras.applications import *
# from keras.optimizers import Adam

# # Data
# # X = input images [num, row, cols, channel]
# # y = input controls 2D matrix [throttle value, steering angle]

# # resolution for images (row, col, channel)
# image_dims = (1280,720,3)

# # get pre-trained model
# xception = ResNet50(include_top=False, input_shape=image_dims)

# # add last layers
# layer = Flatten()(xception.output)
# layer = Dense(1024, activation='relu')(layer)
# layer = Dropout(0.5)(layer)
# layer = Dense(1024, activation='relu')(layer)
# layer = Dropout(0.5)(layer)
# layer = Dense(1024, activation='relu')(layer)
# layer = Dropout(0.5)(layer)

# # output controls. it will be Linear function
# output_layer = Dense(2)(layer)

# # compile the model
# model = Model(xception.input, output_layer)
# model.load_weights('model/trained_model.h5')

# adam = Adam(lr=1e-8, decay=0.5)
# model.compile(optimizer=adam, loss='mse')
# print('Model loaded!')

# def predictControls(image):
#     return model.predict(x=image)

In [ ]:
from __future__ import print_function

import argparse
import logging
import random
import sys
import time

from carla.client import make_carla_client
from carla.sensor import Camera
from carla.settings import CarlaSettings
from carla.tcp import TCPConnectionError


def run_carla_client(host, port, autopilot_on, save_images_to_disk, image_filename_format, settings_filepath):
    number_of_episodes = 5
    frames_per_episode = 300

    with make_carla_client(host, port) as client:
        print('CarlaClient connected')

        for episode in range(0, number_of_episodes):
            # Start a new episode.

            # Create a CarlaSettings object. This object is a wrapper around
            # the CarlaSettings.ini file. Here we set the configuration we
            # want for the new episode.
            settings = CarlaSettings()
            settings.set(
                SynchronousMode=True,
                SendNonPlayerAgentsInfo=True,
                NumberOfVehicles=50,
                NumberOfPedestrians=100,
                WeatherId=random.choice([1, 3, 7, 8, 14]))
            settings.randomize_seeds()

            # Now we want to add a couple of cameras to the player vehicle.
            # We will collect the images produced by these cameras every
            # frame.

            # The default camera captures RGB images of the scene.
            settings.add_sensor(Camera('CameraRGB', image_size=(1280, 720), position=(8, 0, 140), rotation=(-12, 0, 0)))

            scene = client.load_settings(settings)

            # Choose one player start at random.
            player_start = 0

            # Notify the server that we want to start the episode at the
            # player_start index. This function blocks until the server is ready
            # to start the episode.
            print('Starting new episode...')
            client.start_episode(player_start)

            # Iterate every frame in the episode.
            for frame in range(frames_per_episode):

                # Read the data produced by the server this frame.
                measurements, sensor_data = client.read_data()

                # Save the images to disk if requested.
                if save_images_to_disk:
                    for name, image in sensor_data.items():
                        image.save_to_disk(image_filename_format.format(episode, name, frame))

                # Now we have to send the instructions to control the vehicle.
                # If we are in synchronous mode the server will pause the
                # simulation until we send this control.

                # get control instructions from model
                with image as sensor_data['CameraRGB']:
                    steer, throttle = predictControls(image)

                print(steer, throttle)

                # send control to simulator
                client.send_control(
                    steer=steer,
                    throttle=throttle,
                    brake=0.0,
                    hand_brake=False,
                    reverse=False)


def main():
    argparser = argparse.ArgumentParser(description=__doc__)
    argparser.add_argument(
        '-v', '--verbose',
        action='store_true',
        dest='debug',
        help='print debug information')
    argparser.add_argument(
        '--host',
        metavar='H',
        default='localhost',
        help='IP of the host server (default: localhost)')
    argparser.add_argument(
        '-p', '--port',
        metavar='P',
        default=2000,
        type=int,
        help='TCP port to listen to (default: 2000)')
    argparser.add_argument(
        '-a', '--autopilot',
        action='store_true',
        help='enable autopilot')
    argparser.add_argument(
        '-i', '--images-to-disk',
        action='store_false',
        help='save images to disk')
    argparser.add_argument(
        '-c', '--carla-settings',
        metavar='PATH',
        default=None,
        help='Path to a "CarlaSettings.ini" file')

    args = argparser.parse_args()

    log_level = logging.DEBUG if args.debug else logging.INFO
    logging.basicConfig(format='%(levelname)s: %(message)s', level=log_level)

    logging.info('listening to server %s:%s', args.host, args.port)

    while True:
        try:

            run_carla_client(
                host=args.host,
                port=args.port,
                autopilot_on=args.autopilot,
                save_images_to_disk=args.images_to_disk,
                image_filename_format='_images/episode_{:0>3d}/{:s}/image_{:0>5d}.png',
                settings_filepath=args.carla_settings)

            print('Done.')
            return

        except TCPConnectionError as error:
            logging.error(error)
            time.sleep(1)
        except Exception as exception:
            logging.exception(exception)
            sys.exit(1)


if __name__ == '__main__':

    try:
        main()
    except KeyboardInterrupt:
        print('\nCancelled by user. Bye!')
